# BPO

In [1]:
from koselleck import *

In [2]:
to_sents('Tbe Hiftory',correct_ocr=True)

,i_para,i_sent,i_word,tok,tokl,tokl_mod,tokl_ocr,is_punct,is_recog,is_stopword
i_tok,,,,,,,,,,
0,0,0,0,Tbe,tbe,tbe,the,0,1,0
1,0,0,0,,,,,1,0,0
2,0,0,1,Hiftory,hiftory,hiftory,history,0,1,0


In [3]:
# get_wordlist??

In [39]:
def iter_skipgrams(path_txt, n=25,max_len_char=1000000,max_yield=10000):
    try:
        if os.path.exists(path_txt):
            with open(path_txt) as f:
                txt=''.join(ln for ln in f if not ln.startswith('#'))
        else:
            txt=path_txt
        txt=fix_text(txt[:max_len_char])
        df=to_sents(txt)
        df=df[df.is_recog==True]
        df=df[df.is_punct==False]
        df['i_psg']=divide_by_sent_windows(df,nmin=n)
        done=0
        for ipsg,ipsgdf in df.groupby('i_psg'):
            if ipsgdf.is_recog.sum()>=n:
                yield ' '.join(ipsgdf.tokl_ocr).replace('  ',' ')
                done+=1
                if done>=max_yield: break
    except Exception:
        pass

In [40]:
next(iter_skipgrams(C.t.path_txt))

'three books for the english teacher dr manual of english prosody published by messrs macmillan will be found useful to those who believe that children should be led to appreciate the music of poetry'

In [41]:
# next(iter_skipgrams('The hiftory of fable'))

In [42]:
def get_skipgrams(*x,**y): return list(iter_skipgrams(*x,**y))

In [43]:
# get_skipgrams(C.meta.path_txt[0])

In [44]:
def iter_skipgrams_paths(paths_txt,num_proc=1,**attrs):
    for skips in pmap_iter(
        get_skipgrams,
        paths_txt,
        num_proc=num_proc,
        **attrs
    ):
        yield from skips

In [45]:
next(iter_skipgrams_paths(C.meta.path_txt))

Mapping get_skipgrams() [x1]:   0%|          | 0/50449 [00:01<?, ?it/s]


'ana juice or umpire grapes is british vermouth and wine of which we can all be justly proud inspiration to the creation of good vermouth belongs special flair'

In [46]:
def periodize(y):
    if 1700<=y<1770: return '1700-1770'
    if 1770<=y<1830: return '1770-1830'
    if 1830<=y<1900: return '1830-1900'
    

In [48]:
def do_gen_skipgrams_periods(pdf,num_proc=1):
    period=pdf.period[0]
    ofn=os.path.join(PATH_SKIPGRAMS_YR,f'data.skipgrams.bpo.{period}.txt')
    if os.path.exists(ofn): return pd.DataFrame()
    with open(ofn,'w') as of:
        for skp in iter_skipgrams_paths(pdf.sample(frac=1).path_txt, num_proc=num_proc, progress=False, desc=f'Saving {os.path.basename(ofn)}'):
            if skp:
                of.write(skp+'\n')
    return pd.DataFrame()

def gen_skipgrams_periods(meta,num_proc=1,**y):
    meta=meta.query('1680<=year<1980')
    meta['period']=meta.year//1*1
    pmap_groups(do_gen_skipgrams_periods, meta.groupby('period'), num_proc=num_proc, desc='Saving skipgrams', **y)

In [ ]:
gen_skipgrams_periods(C.meta, num_proc=7)

Saving skipgrams [x7]:  92%|█████████▏| 266/289 [5:07:00<05:14, 13.68s/it]   